In [20]:
import pandas as pd
import yfinance as yf
import datetime as dt
import streamlit as st
from datetime import datetime
import os
from operator import itemgetter, attrgetter
import numpy as np

import streamlit as st
import plotly.express as px



import warnings    
warnings.filterwarnings("ignore")





In [21]:
#######################################################################################################
                            ######### Carga do Dataframe ##########
#######################################################################################################
file = '/Outputs/MinhasCotacoes.xlsx'
caminho_atual = os.getcwd()
file = caminho_atual + file

In [22]:
minhasacoes = pd.read_excel(file, sheet_name='Cotação das Ações')
minhasacoes

,SETORECONOMICO,SUBSETOR,SEGMENTO,EMPRESA,TITULO,ACAO,QTDECOTAS,VALORCOMPRA,DATACOMPRA,DATACOTACAO,VALORABERTURA,MAIORCOTACAO,MENORCOTACAO,VALORFECHAMENTO,VALORAJUSTADO,DATAPROCESSAMENTO,VALORDIVIDENDO,FLAG_DIVIDENDOS,ANOCOTACAO,DIVIDENDOSANO
0,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-06,17.60,17.75,17.49,17.64,14.48,2024-06-10,0.0,Não,2019,Sim
1,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-07,17.70,17.78,17.58,17.61,14.46,2024-06-10,0.0,Não,2019,Sim
2,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-10,17.61,17.90,17.60,17.65,14.49,2024-06-10,0.0,Não,2019,Sim
3,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-11,17.68,17.95,17.59,17.72,14.55,2024-06-10,0.0,Não,2019,Sim
4,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-12,17.72,17.79,17.43,17.68,14.51,2024-06-10,0.0,Não,2019,Sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46002,Consumo Cíclico,Comércio Varejista,Eletrodomésticos,CASAS BAHIA,VIAVAREJO ONNM,BHIA3,249,10.23,2020-03-19,2024-06-03,7.03,7.12,6.88,6.98,6.98,2024-06-10,0.0,Não,2024,Não
46003,Consumo Cíclico,Comércio Varejista,Eletrodomésticos,CASAS BAHIA,VIAVAREJO ONNM,BHIA3,249,10.23,2020-03-19,2024-06-04,6.97,6.98,6.35,6.35,6.35,2024-06-10,0.0,Não,2024,Não
46004,Consumo Cíclico,Comércio Varejista,Eletrodomésticos,CASAS BAHIA,VIAVAREJO ONNM,BHIA3,249,10.23,2020-03-19,2024-06-05,6.35,6.48,6.18,6.27,6.27,2024-06-10,0.0,Não,2024,Não
46005,Consumo Cíclico,Comércio Varejista,Eletrodomésticos,CASAS BAHIA,VIAVAREJO ONNM,BHIA3,249,10.23,2020-03-19,2024-06-06,6.27,6.79,6.27,6.68,6.68,2024-06-10,0.0,Não,2024,Não


In [23]:
# função para selecionar a quantidade de linhas do dataframe
def mostra_qntd_linhas(dataframe):

    qntd_linhas = st.sidebar.slider('Selecione a quantidade de linhas que deseja mostrar na tabela', min_value = 1, max_value = len(dataframe), step = 1)

    #st.write(dataframe.head(qntd_linhas).style.format(subset = ['Valor'], formatter="{:.2f}"))

In [24]:
def retorna_ultima_cotacao_acao(df):
    latest_quotes = df.loc[df.groupby('ACAO')['DATACOTACAO'].idxmax()]
    latest_quotes = latest_quotes[['ACAO', 'DATACOTACAO', 'QTDECOTAS', 'VALORFECHAMENTO']].drop_duplicates()
    latest_quotes['VALORTOTALATUAL'] = latest_quotes['QTDECOTAS'] * latest_quotes['VALORFECHAMENTO']
    return latest_quotes

In [25]:
def create_dataframe(coluna):
    df = minhasacoes[[coluna, 'QTDECOTAS', 'VALORCOMPRA', 'ACAO']].drop_duplicates()
    df['VALORTOTALCOMPRA'] = df['QTDECOTAS'] * df['VALORCOMPRA']
    df = df.groupby([coluna, 'ACAO']).agg({
        'QTDECOTAS': np.sum,
        'VALORTOTALCOMPRA': np.sum
    }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
    return df

In [26]:
def consolidate_details(df, latest_quotes):
    df = pd.merge(df, latest_quotes[['ACAO', 'DATACOTACAO', 'VALORTOTALATUAL']], on='ACAO', how='inner')
    df = df.drop_duplicates().sort_values(by='ACAO').reset_index(drop=True)
    return df

In [27]:
def group_by(df,coluna):
    df = df.groupby(coluna).agg({
        'QTDECOTAS': np.sum,
        'VALORTOTALCOMPRA': np.sum,
        'VALORTOTALATUAL': np.sum
    }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
    return df

In [28]:
def update_performance(df):
    df['RENDIMENTO(%)'] = ((df['VALORTOTALATUAL'] / df['VALORTOTALCOMPRA'] - 1) * 100).round(2)
    df['PREJUIZO(%)'] = ((df['VALORTOTALCOMPRA'] / df['VALORTOTALATUAL'] - 1) * 100).round(2)
    df.loc[df['VALORTOTALATUAL'] < df['VALORTOTALCOMPRA'], 'RENDIMENTO(%)'] = 0
    df.loc[df['VALORTOTALATUAL'] >= df['VALORTOTALCOMPRA'], 'PREJUIZO(%)'] = 0
    return df

In [29]:
def agupamento(agrupamento, df):
    coluna = agrupamento
    ult_cot = retorna_ultima_cotacao_acao(df)
    dtf = create_dataframe(coluna)
    dtf = consolidate_details(dtf, ult_cot)
    dtf = group_by(dtf, coluna)
    dtf = update_performance(dtf)
    return dtf

In [30]:
st = agupamento('SETORECONOMICO', minhasacoes)
subst = agupamento('SUBSETOR', minhasacoes)
sg = agupamento('SEGMENTO', minhasacoes)
emp = agupamento('EMPRESA', minhasacoes)

In [31]:
fig = px.bar(st, x="VALORTOTALCOMPRA", y="SETORECONOMICO", orientation='h')
fig.show()

In [32]:
import plotly.express as px

df = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(df, x='year', y='pop',
             hover_data=['lifeExp', 'gdpPercap'], color='lifeExp',
             labels={'pop':'population of Canada'}, height=400)
fig.show()

In [33]:
df = minhasacoes[minhasacoes['ACAO'] == 'ABEV3']
df = df.rename(columns={'VALORAJUSTADO':'Cotação no Período','MAIORCOTACAO':'Máxima Cotação Período', 'MENORCOTACAO':'Menor Cotação Período', 'VALORCOMPRA': 'Valor de Compra'})

In [34]:
df.loc[df.index.max(), "Cotação no Período"]

11.56

In [35]:
ult_cot = retorna_ultima_cotacao_acao(df)
ult_cot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 1243 to 1243
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ACAO             1 non-null      object        
 1   DATACOTACAO      1 non-null      datetime64[ns]
 2   QTDECOTAS        1 non-null      int64         
 3   VALORFECHAMENTO  1 non-null      float64       
 4   VALORTOTALATUAL  1 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 48.0+ bytes


In [36]:
# Função para retornar a ultima cotação da ação
def retorna_ultima_cotacao_acao(df):
    latest_quotes = df.loc[df.groupby('ACAO')['DATACOTACAO'].idxmax()]
    latest_quotes = latest_quotes[['ACAO', 'DATACOTACAO', 'QTDECOTAS', 'VALORFECHAMENTO']].drop_duplicates()
    latest_quotes['VALORTOTALATUAL'] = latest_quotes['QTDECOTAS'] * latest_quotes['VALORFECHAMENTO']
    return latest_quotes

# Função para retornar um dataframe agrupado
def create_dataframe(coluna):
    df = minhasacoes[[coluna, 'QTDECOTAS', 'VALORCOMPRA', 'ACAO']].drop_duplicates()
    df['VALORTOTALCOMPRA'] = df['QTDECOTAS'] * df['VALORCOMPRA']
    df = df.groupby([coluna, 'ACAO']).agg({
        'QTDECOTAS': np.sum,
        'VALORTOTALCOMPRA': np.sum
    }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
    return df

# Função para consolidar o dataframe
def consolidate_details(df, latest_quotes):
    df = pd.merge(df, latest_quotes[['ACAO', 'DATACOTACAO', 'VALORTOTALATUAL']], on='ACAO', how='inner', validate="many_to_many")
    df = df.drop_duplicates().sort_values(by='ACAO').reset_index(drop=True)
    return df

# Função para agrupar o dataframe
def group_by(df,coluna):
    df = df.groupby(coluna).agg({
        'QTDECOTAS': np.sum,
        'VALORTOTALCOMPRA': np.sum,
        'VALORTOTALATUAL': np.sum
    }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
    return df

# Função para atualizar percentual de rendimento e prejuizo
def update_performance(df):
    df['RENDIMENTO(%)'] = ((df['VALORTOTALATUAL'] / df['VALORTOTALCOMPRA'] - 1) * 100).round(2)
    df['PREJUIZO(%)'] = ((df['VALORTOTALCOMPRA'] / df['VALORTOTALATUAL'] - 1) * 100).round(2)
    df['RETORNO(%)'] = ((df['VALORTOTALATUAL'] / df['VALORTOTALCOMPRA'] - 1) * 100).round(2)
    df.loc[df['VALORTOTALATUAL'] < df['VALORTOTALCOMPRA'], 'RENDIMENTO(%)'] = 0
    df.loc[df['VALORTOTALATUAL'] >= df['VALORTOTALCOMPRA'], 'PREJUIZO(%)'] = 0
    return df

# Função que centraliza as funções 
def agupamento(agrupamento, df):
    coluna = agrupamento
    ult_cot = retorna_ultima_cotacao_acao(df)
    dtf = create_dataframe(coluna)
    dtf = consolidate_details(dtf, ult_cot)
    dtf = group_by(dtf, coluna)
    dtf = update_performance(dtf)
    return dtf

# **************************************************************************************************

st = agupamento('SETORECONOMICO', minhasacoes)
subst = agupamento('SUBSETOR', minhasacoes)
sg = agupamento('SEGMENTO', minhasacoes)
emp = agupamento('EMPRESA', minhasacoes)

# **************************************************************************************************

In [37]:
st

,SETORECONOMICO,QTDECOTAS,VALORTOTALCOMPRA,VALORTOTALATUAL,RENDIMENTO(%),PREJUIZO(%),RETORNO(%)
0,"Petróleo, Gás e Biocombustíveis",464,9465.68,17552.66,85.43,0.00,85.43
1,Financeiro,486,7675.54,5381.16,0.00,42.64,-29.89
2,Consumo Cíclico,488,6933.29,2284.96,0.00,203.43,-67.04
3,Saúde,213,5202.53,1547.21,0.00,236.25,-70.26
4,Comunicações,2740,3251.00,3288.80,1.16,0.00,1.16
5,Materiais Básicos,247,3025.79,4468.28,47.67,0.00,47.67
6,Consumo não Cíclico,101,2001.04,1286.43,0.00,55.55,-35.71
7,Bens Industriais,107,1293.33,806.01,0.00,60.46,-37.68
8,Utilidade Pública,100,1251.00,1121.00,0.00,11.60,-10.39
9,Tecnologia da Informação,50,1023.50,211.00,0.00,385.07,-79.38
